In [1]:
import pandas as pd
import json

In [9]:
with open('/tmp/pycharm_project_366/config.json') as f:
    config = json.load(f)

In [18]:
def get_people_with_disease(df, all_diseased_column, disease_code_pattern):
    people_with_disease_df = df[all_diseased_column.str.contains(disease_code_pattern)]

    return people_with_disease_df

In [10]:
test_group_df = pd.read_csv(config['test_path'], low_memory=False)
test_group_df

,eid,34-0.0,84-0.0,874-0.0,884-0.0,894-0.0,904-0.0,914-0.0,1001-0.0,1070-0.0,...,23417-0.0,23418-0.0,23420-0.0,23421-0.0,23422-0.0,23526-0.0,23527-0.0,23528-0.0,Diagnoses,Label
0,1002706,1961.0,NaN,45.0,0.0,NaN,0.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"G551, M511, M519, M545, M5457, M7915, N920, Z5...",0
1,1060573,1940.0,NaN,240.0,7.0,180.0,3.0,120.0,NaN,3.0,...,1.6842,1.5641,0.28381,0.67221,0.47619,1.2646,0.92185,0.34273,"G551, H268, H919, I10, K579, K623, K628, K635,...",0
2,1053496,1940.0,NaN,30.0,4.0,60.0,0.0,NaN,NaN,5.0,...,1.3799,0.74993,0.28523,0.5423,0.2195,0.95396,0.70152,0.25244,"D128, E780, I48, I639, K621, K635, -1, -1, -1,...",0
3,1069209,1945.0,NaN,60.0,1.0,60.0,0.0,NaN,NaN,1.0,...,1.6023,1.3042,0.25467,0.64997,0.36911,1.1623,0.8618,0.30051,"H258, K439, K590, R194, -1, -1, -1, -1, -1, -1...",0
4,1064821,1946.0,NaN,15.0,3.0,60.0,3.0,10.0,NaN,5.0,...,1.2505,1.0868,0.2947,0.44826,0.32233,0.86331,0.6464,0.21691,"F419, H409, K219, K590, M1300, M139, M169, M20...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100519,6003239,1968,NaN,15.0,4.0,45.0,4.0,45.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1...",0
100520,6002899,1960,NaN,30.0,2.0,30.0,2.0,30.0,NaN,2.0,...,1.0722,1.0485,0.23734,0.39342,0.28537,0.75261,0.54702,0.20559,"H026, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, ...",0
100521,6013344,1956,NaN,120.0,7.0,60.0,7.0,240.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"E119, E669, H023, I10, I252, I259, R060, R072,...",2
100522,6001796,1941,NaN,50.0,-1.0,NaN,-1.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"H269, J181, J22, J439, J440, J441, J841, J920,...",0


In [13]:
train_group_df = pd.read_csv(config['train_path'], low_memory=False)
train_group_df

,eid,34-0.0,84-0.0,874-0.0,884-0.0,894-0.0,904-0.0,914-0.0,1001-0.0,1070-0.0,...,23417-0.0,23418-0.0,23420-0.0,23421-0.0,23422-0.0,23526-0.0,23527-0.0,23528-0.0,Diagnoses,Label
0,1042612,1948.0,NaN,30.0,7.0,60.0,4.0,30.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"E039, E669, H400, I48, I481, M159, M2322, -1, ...",0
1,1085780,1943.0,2008.0,20.0,0.0,NaN,0.0,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"E039, E871, I10, R55, R730, Y543, Z602, Z922, ...",0
2,1056843,1965.0,NaN,15.0,3.0,20.0,3.0,10.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"B977, D071, F329, K628, N908, Z090, Z532, Z860...",0
3,1020608,1944.0,NaN,30.0,3.0,10.0,5.0,10.0,NaN,2.0,...,1.3961,2.0498,0.25361,0.53007,0.65695,1.2128,0.89636,0.31648,"I10, K221, K317, K449, -1, -1, -1, -1, -1, -1,...",0
4,1033909,1947.0,NaN,NaN,3.0,30.0,0.0,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"E039, R590, -1, -1, -1, -1, -1, -1, -1, -1, -1...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72459,6023669,1957,NaN,10.0,2.0,10.0,3.0,10.0,NaN,3.0,...,1.117,1.2532,0.21467,0.41503,0.36378,0.91329,0.70299,0.2103,"A09, D481, E119, E780, I10, I639, L721, Z867, ...",2
72460,6023754,1951,NaN,45.0,0.0,NaN,0.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"E119, E780, I10, K580, R194, R55, Z538, -1, -1...",2
72461,6001401,1938,1999.0,15.0,4.0,45.0,0.0,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"C252, C509, C787, E039, E042, E059, E119, E222...",1
72462,6003146,1942,NaN,15.0,0.0,NaN,0.0,NaN,NaN,4.0,...,1.1325,1.3802,0.19679,0.40907,0.3865,0.74372,0.55458,0.18913,"B356, B972, B99, C259, D136, D225, D350, D377,...",1


In [12]:
total_people_in_train = train_group_df.shape[0]
people_with_pancreatic_cancer_in_train = train_group_df[train_group_df['Label'] == '1'].shape[0]
people_with_diabetes_in_train = train_group_df[train_group_df['Label'] == '2'].shape[0]
total_people_in_test = test_group_df.shape[0]
people_with_pancreatic_cancer_in_test = test_group_df[test_group_df['Label'] == '1'].shape[0]
people_with_diabetes_in_test = test_group_df[test_group_df['Label'] == '2'].shape[0]
print(f'Total people in train: {total_people_in_train}')
print(f'Number of people with diabetes in train: {people_with_diabetes_in_train}')
print(f'Number of people with pancreatic cancer in train: {people_with_pancreatic_cancer_in_train}')
print(f'Total people in test: {total_people_in_test}')
print(f'Number of people with diabetes in test: {people_with_diabetes_in_test}')
print(f'Number of people with pancreatic cancer in test: {people_with_pancreatic_cancer_in_test}')

Total people in train: 72464
Number of people with diabetes in train: 35246
Number of people with pancreatic cancer in train: 1613
Total people in test: 100524
Number of people with diabetes in test: 8470
Number of people with pancreatic cancer in test: 391


In [5]:
df_test.iloc[:,270]

In [10]:
df_train = pd.read_csv('/tmp/pycharm_project_366/train_data.csv')
df_train

In [9]:
set(df_test.columns) - set(config['features_code_list'])

In [10]:
'41270-0.99.1' in config['features_code_list']

In [11]:
features = pd.read_csv('/home/ofeksh2@mta.ac.il/biobank/fields672220.ukb')

In [21]:
'21028-0.0.1' in config['features_code_list']